<a href="https://colab.research.google.com/github/deethereal/NN/blob/main/NLP%5CNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np, pandas as pd
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

onehots={}
onehots['cat']=np.array([1,0,0,0])
onehots['the']=np.array([0,1,0,0])
onehots['dog']=np.array([0,0,1,0])
onehots['sat']=np.array([0,0,0,1])

sentence=['the','cat','sat','sat','dog']

x=onehots[sentence[0]]
for i in range(1,len(sentence)):
  x+=onehots[sentence[i]]
print(str(x))

[1 1 1 2]


In [8]:
#lines=csv.DictReader("/content/gdrive/My Drive/IMDB.csv/",delimiter=',')
%cd "/content/drive/My Drive/"
lines=pd.read_csv('IMDB.csv', sep=',')
for line in lines:
  raw_reviews, raw_labels= lines['review'], lines['sentiment']
tokens=list(map(lambda x:set(x.split(" ")),raw_reviews))
vocab=set()
for sent in tokens:
  for word in sent:
    if len(word)>0:
      vocab.add(word)
vocab=list(vocab)
word2index={}
for i, word in enumerate(vocab):
  word2index[word]=i
input_dataset=list()
for sent in tokens:
  sent_indices = list()
  for word in sent:
    try:
      sent_indices.append(word2index[word])
    except:
      ""
  input_dataset.append(list(set(sent_indices)))
target_dataset=list()
for label in raw_labels:
  if label == 'positive':
    target_dataset.append(1)
  else:
    target_dataset.append(0)


/content/drive/My Drive


In [ ]:
print(len(vocab))

439838


In [ ]:
print(len(input_dataset[2]),len(raw_reviews[2].split()))
print(raw_reviews[2])

121 166
I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.


In [ ]:
S=len(input_dataset)
print(S)

50000


In [12]:
np.random.seed(1)
def sigmoid(x):
  return 1/(1+np.exp(-x))
learning_rate, niter, hidden_size= 0.01, 35, 150
W01=0.2*np.random.random((len(vocab),hidden_size))-0.1
W12=0.2*np.random.random((hidden_size,1))-0.1
correct, total = 0,0
for iter in range(niter):
  for i in range(len(input_dataset)-2000):
    x,y=(input_dataset[i],target_dataset[i])
    l1=sigmoid(np.sum(W01[x],axis=0))
    l2=sigmoid(np.dot(l1,W12))
  

    l2_delta=l2-y
    l1_delta=l2_delta.dot(W12.T)

    W01[x]-=l1_delta*learning_rate
    W12-=np.outer(l1,l2_delta)*learning_rate

    if (np.abs(l2_delta) < 0.5):
      correct+=1
    total+=1
    """if (i% 10 == 9):
      progress=str(i/float(len(input_dataset)-26000))
      print("Iter: {} Progress: {}.{} TrainingAccuracy: {}".format(iter,progress[2:4],progress[4:6],(correct/float(total))))"""
correct, total = 0,0
for i in range(len(input_dataset)-2000,len(input_dataset)):
  x,y=(input_dataset[i],target_dataset[i])
  l1=sigmoid(np.sum(W01[x],axis=0))
  l2=sigmoid(np.dot(l1,W12))
    
  if np.abs(l2-y)<0.1:
    correct+=1
  total+=1
print('Test Accuracy: {}%'.format(100*correct/float(total)))

Test Accuracy: 81.75%


In [ ]:
def nn(x):
  l1=sigmoid(np.sum(W01[x],axis=0))
  l2=sigmoid(np.dot(l1,W12))
  print(l2)
  print("The nature of review is", end=" ")
  print("positive") if l2>0.5 else print("negative")
def proccecing(x):
  x=list(set(x.split(" ")))
  sent_indices = list()
  for word in x:
    try:
      sent_indices.append(word2index[word])
    except:
      ""
  return list(set(sent_indices))



In [ ]:
mr1="The film is very boring, I kept waiting for it to start !"
nn(proccecing(mr1))
mr2="I really liked this film, almost everything is great in it: the acting, the plot and the special effects !"
nn(proccecing(mr2))



[0.00367403]
The nature of review is negative
[0.99998366]
The nature of review is positive


***Эта архитектура ищет прямую корреляцию между входом и выходом***  
Поэтому получается так:

In [ ]:
test1='good film not bad'
test2='bad film not good'
nn(proccecing(test1))
nn(proccecing(test2))

[3.11175849e-05]
The nature of review is negative
[3.11175849e-05]
The nature of review is negative


In [59]:
from collections import Counter
def similiar(target='beautiful'):
  target_index=word2index[target]
  scores=Counter()
  for word, index in word2index.items():
    raw_diff=W01[index]-W01[target_index]
    scores[word]=-np.sqrt(np.sum((raw_diff)**2))
  return scores.most_common(10)
def sigmoid(x):
  return 1/(1+np.exp(-x))

In [35]:
print(similiar('Marvel'))

[('Marvel', -0.0), ('ledger:', -0.37299359811339494), ('Marine(Naval', -0.37590073475230473), ('Peirce', -0.37950251177209005), ('"Heaven\'s', -0.37995265294910785)]


In [30]:
np.random.seed(1)
%cd "/content/drive/My Drive/"
lines=pd.read_csv('IMDB.csv', sep=',')
for line in lines:
  raw_reviews, raw_labels= lines['review'], lines['sentiment']
tokens=list(map(lambda x:set(x.split(" ")),raw_reviews))
wordcnt= Counter() 
for sent in tokens:
  for word in sent:
    wordcnt[word]-=1
vocab=list(set(map(lambda x:x[0],wordcnt.most_common())))


word2index={}
for i,word in enumerate(vocab):
  word2index[word]=i

concatenated = list()
input_dataset = list()
for sent in tokens:
  sent_indices=list()
  for word in sent:
    try:
      sent_indices.append(word2index[word])
      concatenated.append(word2index[word])
    except:
      ""
  input_dataset.append(sent_indices)
concatenated=np.array(concatenated)

np.random.shuffle(input_dataset)
print("Data is ready!")

/content/drive/My Drive
Data is ready!


In [69]:
learning_rate, hidden_size, niter = 0.05,50,2
window,negative = 2,5

W01=0.2*(np.random.random((len(vocab),hidden_size))-0.5)
W12=0.2*(np.random.random((len(vocab),hidden_size,)) -0.5)
l2_target=np.zeros(negative+1)
l2_target[0]=1


for rev_i, review in enumerate(input_dataset*niter):
  for target_i in range(len(review)):
    target_sample=[review[target_i]]+list(concatenated[(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])  #выбираем случайное подмонжество


    left_context=review[max(0,target_i-window):target_i]
    right_context=review[target_i+1:min(len(review),target_i+window)]


    l1=np.mean(W01[left_context+right_context], axis=0)
    l2=sigmoid(l1.dot(W12[target_sample].T))

    l2_delta=l2-l2_target
    l1_delta=l2_delta.dot(W12[target_sample])

    W01[left_context+right_context]-=l1_delta*learning_rate
    W12[target_sample]-=np.outer(l2_delta,l1)*learning_rate

  print('Progress: {} '.format(rev_i/float(len(input_dataset))))  
print(similiar('terrible'))


Выходные данные были обрезаны до нескольких последних строк (5000).
Progress: 1.90002 
Progress: 1.90004 
Progress: 1.90006 
Progress: 1.90008 
Progress: 1.9001 
Progress: 1.90012 
Progress: 1.90014 
Progress: 1.90016 
Progress: 1.90018 
Progress: 1.9002 
Progress: 1.90022 
Progress: 1.90024 
Progress: 1.90026 
Progress: 1.90028 
Progress: 1.9003 
Progress: 1.90032 
Progress: 1.90034 
Progress: 1.90036 
Progress: 1.90038 
Progress: 1.9004 
Progress: 1.90042 
Progress: 1.90044 
Progress: 1.90046 
Progress: 1.90048 
Progress: 1.9005 
Progress: 1.90052 
Progress: 1.90054 
Progress: 1.90056 
Progress: 1.90058 
Progress: 1.9006 
Progress: 1.90062 
Progress: 1.90064 
Progress: 1.90066 
Progress: 1.90068 
Progress: 1.9007 
Progress: 1.90072 
Progress: 1.90074 
Progress: 1.90076 
Progress: 1.90078 
Progress: 1.9008 
Progress: 1.90082 
Progress: 1.90084 
Progress: 1.90086 
Progress: 1.90088 
Progress: 1.9009 
Progress: 1.90092 
Progress: 1.90094 
Progress: 1.90096 
Progress: 1.90098 
Progress: 

In [70]:
print(similiar('beautiful'))

[('beautiful', -0.0), ('through.', -3.257620791899391), ('VHS', -3.3630036060581245), ('funny', -3.536872647245672), ('amateurish', -3.5721595381322744), ('/>My', -3.595484517253307), ('line,', -3.6373485684248563), ('colour', -3.687585749205025), ('running', -3.765119284817027), ('mature', -3.769495493009992)]


In [71]:
def analogy(pos=[ 'terrible','good' ], negative = ['bad'] ):
  norms=np.sum(W01*W01,axis=1)
  norms.resize(norms.shape[0],1)

  normed_W=W01*norms
  query_vect = np.zeros(len(W01[0]))
  for word in pos:
    query_vect+=normed_W[word2index[word]]
  for word in negative:
    query_vect-=normed_W[word2index[word]]
  scores = Counter()
  for word, index in word2index.items():
    raw_diff= W01[index]-query_vect
    sqr=raw_diff*raw_diff
    scores[word]=np.sqrt(np.sum(sqr))
  return scores.most_common(10)[1:]
print(analogy())

[('loved', 1190.6085253997278), ('his', 1190.4106331982223), ('well,', 1190.3229345631055), ('time,', 1190.3140537579422), ('follows', 1190.2941402243123), ('left', 1190.239690607444), ('"You', 1190.1201370996573), ('either', 1190.1125348934206), ('series,', 1190.070524872778)]
